In [8]:
from lib.Analysis import Analysis
import numpy as np
analysis = Analysis()

In [9]:
import pandas as pd
import datetime
all_data = analysis.get_all_json_files()
dataframe = []

for filename in all_data:
  locais = all_data[filename]
  for local in locais:
    dataframe.append({
      'nome': local.nome,
      'preco': local.preco,
      'time': datetime.datetime.fromtimestamp(local.time),
      'owner_cresci': local.owner.cresci,
      'owner_nome': local.owner.nome,
      'owner_id': local.owner.cresci,
      'celular': local.owner.celular,
      'telefone': local.owner.telefone,
      'cep': local.cep
    })

[<entities.Empreendimento.Local object at 0x000001EEFEED7950>, <entities.Empreendimento.Local object at 0x000001EEFF2D3C50>, <entities.Empreendimento.Local object at 0x000001EEFE7F7590>, <entities.Empreendimento.Local object at 0x000001EE9C0352E0>, <entities.Empreendimento.Local object at 0x000001EE9C0351C0>, <entities.Empreendimento.Local object at 0x000001EE9C035310>, <entities.Empreendimento.Local object at 0x000001EE9C035220>, <entities.Empreendimento.Local object at 0x000001EE9C035160>, <entities.Empreendimento.Local object at 0x000001EE9C035100>, <entities.Empreendimento.Local object at 0x000001EE9C0350A0>, <entities.Empreendimento.Local object at 0x000001EE9C035040>, <entities.Empreendimento.Local object at 0x000001EE9C034FE0>, <entities.Empreendimento.Local object at 0x000001EE9C034F80>, <entities.Empreendimento.Local object at 0x000001EE9C034F20>, <entities.Empreendimento.Local object at 0x000001EE9C034EC0>, <entities.Empreendimento.Local object at 0x000001EE9C034E60>, <entiti

In [10]:
df = pd.DataFrame(dataframe)
print(df)

                                                 nome    preco  \
0             Apartamento com 2 Quartos à venda, 82m²  1050000   
1             Apartamento com 2 Quartos à venda, 60m²   184520   
2                    Casa com 3 Quartos à venda, 95m²   485000   
3     Casa de Condomínio com 3 Quartos à venda, 280m²  2490000   
4                         Lote/Terreno à venda, 360m²   645000   
...                                               ...      ...   
7595  Casa de Condomínio com 3 Quartos à venda, 205m²  2150000   
7596                Casa com 4 Quartos à venda, 190m²   585000   
7597                Casa com 5 Quartos à venda, 202m²   800000   
7598  Casa de Condomínio com 3 Quartos à venda, 263m²  1490000   
7599          Apartamento com 2 Quartos à venda, 60m²   245000   

                           time owner_cresci                   owner_nome  \
0    2023-11-04 12:32:17.702668   00000-F-SP              João B. M. Neto   
1    2023-11-04 12:32:17.702668                      

In [11]:
import re

def extrair_tamanho(texto):
    tamanho = re.search(r'(\d+\.?\d*)m²', texto)
    if tamanho:
        return tamanho.group(1)
    return None
df['tamanho'] = df['nome'].apply(extrair_tamanho)

def extrair_numero_quartos(texto):
    numero_quartos = re.search(r'(\d+) Quartos', texto)
    if numero_quartos:
        return int(numero_quartos.group(1))
    return None

df['numero_quartos'] = df['nome'].apply(extrair_numero_quartos)
df['preco_numerica'] = df['preco'].astype(float)
df['tamanho'] = df['tamanho'].astype(float)
df['valor_metro_quadrado'] = df['preco_numerica']/df['tamanho']
df['valor_metro_quadrado'] = df['valor_metro_quadrado'].round(2)
df['tipo'] = df['owner_cresci'].str.split('-').str[1]
df['tipo'] = df['tipo'].str.replace('F', 'Pessoa Física').str.replace('J', 'Pessoa Jurídica')






print(df['tipo'] )



0         Pessoa Física
1                   NaN
2                   NaN
3       Pessoa Jurídica
4         Pessoa Física
             ...       
7595    Pessoa Jurídica
7596      Pessoa Física
7597    Pessoa Jurídica
7598    Pessoa Jurídica
7599    Pessoa Jurídica
Name: tipo, Length: 7600, dtype: object


In [12]:
contagem_tipo= df['tipo'].value_counts()
contagem_tipo

tipo
Pessoa Jurídica    4459
Pessoa Física      1162
Name: count, dtype: int64

In [13]:
df['codigo'] = df['owner_cresci'].str.replace('-', '')
contagem_corretos = df['codigo'].value_counts()
contagem_corretos

codigo
             1979
186520FSP     278
21893JSP      276
027811JSP     261
10179JSP      170
             ... 
39273JSP        1
143999FSP       1
64843FSP        1
21320JSP        1
31476JSP        1
Name: count, Length: 446, dtype: int64

In [14]:
df_sem_duplicatas = df.drop_duplicates(subset='codigo')
df_sem_duplicatas
contagem_corretos = df_sem_duplicatas.groupby('tipo').size().reset_index(name='contagem_corretos')
contagem_corretos

,tipo,contagem_corretos
0,Pessoa Física,168
1,Pessoa Jurídica,277


In [15]:
quartis = df.groupby('tipo')['valor_metro_quadrado'].quantile([0.25,0.5,0.75])
quartis = quartis.reset_index()
quartis.columns = ['tipo', 'quartil', 'valor']
df_pivot = quartis.pivot(index='tipo', columns='quartil', values='valor')
df_pivot['iqr'] = df_pivot[0.75] - df_pivot[0.25]
df_pivot['limit_min'] = df_pivot[0.25] - (1.5)*df_pivot['iqr']
df_pivot['limit_max'] = df_pivot[0.75] + (1.5)*df_pivot['iqr']
#df_pivot['mediana'] = df_pivot[0.75] + (1.5)*df_pivot['iqr']

df_pivot

quartil,0.25,0.5,0.75,iqr,limit_min,limit_max
tipo,,,,,,
Pessoa Física,2291.67,4285.71,6666.67,4375.00,-4270.83,13229.17
Pessoa Jurídica,2738.10,4305.56,6575.34,3837.24,-3017.76,12331.20


In [16]:
alto_valor = df.query('tipo == "Pessoa Física" and valor_metro_quadrado >= 5647.06')
alto_valor_total = alto_valor['codigo'].value_counts()

alto_valorsemduplicadpos = alto_valor.drop_duplicates(subset='codigo')
alto_valor_total

codigo
186520FSP    49
78665FSP     22
83717FSP     20
223682FSP    15
192066FSP    15
             ..
53040FSP      1
175487FSP     1
55916FSP      1
58665FSP      1
98447FSP      1
Name: count, Length: 102, dtype: int64